In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

In [2]:
env = gym.make('StockTradingEnvironment-v0', use_technical_indicators= [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
])

In [3]:
from src.agents.actor_critic.a2c import A2CAgent

In [4]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
                   use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    env.success_threshold =0.05 # 5%
    return env


In [5]:
agent=A2CAgent(environment, epochs=1, actor_learning_rate=0.000025,critic_learning_rate=0.000025)
agent.load()
environment().success_threshold


[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/StockTradingEnvironment/049eb1b62c63b64fe265373bb5f844ba/a2c-actor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/StockTradingEnvironment/049eb1b62c63b64fe265373bb5f844ba/a2c-critic.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)




0.05

In [6]:
agent.hash

'049eb1b62c63b64fe265373bb5f844ba'

In [ ]:
agent.env.mode = "train"
agent.learning_log.episodes = 0
agent.learn(
    timesteps=-1, 
    log_every=1000,
    success_threshold_lookback=1000,
    success_strict=True,
    
)